<a href="https://colab.research.google.com/github/daiki-skm/oreilly-web-optimization/blob/main/chapter6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pymc3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.2/872.2 KB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 49.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for theano-pymc: filename=Theano_PyMC-1.1.2-py3-none-any.whl size=1529963 sha256=5e866d40453dcf56f446a6ef9e4ffab9df69fce3b23a10a2ba09207e1ba0ab96
  Stored in directory: /root/.cache/pip/wheels/0e/41/d2/82c7b771236f987def7fe2e51855cce22b270327f3fedec57c
Successfully built theano-pymc


In [16]:
import numpy as np
import pymc3 as pm

arms = [[0,0],[0,1],[1,0],[1,1]]

class MCMC_GLMTSAgent(object):
  def __init__(self):
    self.counts = [0 for _ in arms]
    self.wins = [0 for _ in arms]
    self.phis = np.array([[arm[0],arm[1],1] for arm in arms]).T
  
  def get_arm(self):
    if 0 in self.counts: return self.counts.index(0)
    with pm.Model() as model:
      w = pm.Normal('w',mu=0,sigma=10,shape=3)
      linpred = pm.math.dot(w,self.phis)
      theta = pm.Deterministic('theta',1/(1+pm.math.exp(-linpred)))
      obs = pm.Binomial('obs',n=self.counts,p=theta,observed=self.wins)
      trace = pm.sample(2000,chains=1)
    sample = pm.sample_posterior_predictive(trace,samples=1,model=model,var_names=[theta])
    return np.argmax(sample['theta'])
  
  def sample(self,arm_index,reward):
    self.counts[arm_index] += 1
    self.wins[arm_index] += reward

In [17]:
class Env(object):
  def p(arm):
    x = arm[0]*0.2+arm[1]*0.8-4
    p = 1/(1+np.exp(-x))
    return p
  
  def react(arm):
    return 1 if np.random.random() < Env.p(arm) else 0
  
  def opt():
    return np.argmax([Env.p(arm) for arm in arms])

In [18]:
np.random.seed(0)
selected_arms = []
earned_rewards = []
n_step = 20
agent = MCMC_GLMTSAgent()
for step in range(n_step):
  arm_index = agent.get_arm()
  for _ in range(50):
    reward = Env.react(arms[arm_index])
    agent.sample(arm_index,reward)
    selected_arms.append(arm_index)
    earned_rewards.append(reward)

theta ~ Deterministic


/usr/local/lib/python3.8/dist-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)


ERROR:pymc3:There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
/usr/local/lib/python3.8/dist-packages/pymc3/sampling.py:1708: UserWarning: samples parameter is smaller than nchains times ndraws, some draws and/or chains may not be represented in the returned posterior predictive sample
  warnings.warn(


KeyError: ignored

In [1]:
class LinUCBAgent(object):
  def __init__(self):
    self.phis = np.array([[arm[0],arm[1],1] for arm in arms]).T
    self.alpha = 1
    self.sigma = 1
    self.A = np.indetity(self.phis.shape[0])
    self.b = np.zeros((self.phis.shape[0],1))
  
  def get_arm(self):
    inv_A = np.linalg.inv(self.A)
    mu = inv_A.dot(self.b)
    S = inv_A
    pred_mean = self.phis.T.dot(mu)
    pred_var = self.phis.T.dot(S).dot(self.phis)
    ucb = pred_mean.T + self.alpha * np.sqrt(np.diag(pred_var))
    return np.argmax(ucb)
  
  def sample(self,arm_index,reward):
    phi = self.phis[:,[arm_index]]
    self.b = self.b + phi * reward / (self.sigma ** 2)
    self.A = self.A + phi.dot(phi.T) / (self.sigma ** 2)

In [3]:
import numpy as np
from matplotlib import pyplot as plt

arms = [[0,0],[0,1],[1,0],[1,1]]
n_iter = 500
n_step = 5000
selected_arms = np.zeros((n_iter,n_step),dtype=int)
earned_rewards = np.zeros((n_iter,n_step),dtype=int)
for it in range(n_iter):
  agent = LinUCBAgent()
  for step in range(n_step):
    arm_index = agent.get_arm()
    reward = Env.react(arms[arm_index])
    agent.sample(arm_index,reward)
    selected_arms[it,step] = arm_index
    earned_rewards[it,step] = reward
plt.plot(np.mean(selected_arms==Env.opt(),axis=0))
plt.xlabel(r'$t$')
plt.ylabel(r'$\mathbb{E}[x(t) = x^*]')
plt.show()

AttributeError: ignored